In [5]:
import os
import pandas as pd
import requests
import json
import time
import dateutil
import datetime
from dateutil.relativedelta import relativedelta
import urllib3
from dotenv import load_dotenv
import logging
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
logging.basicConfig(filename="logs/scraping.log",level=logging.DEBUG)

In [2]:
load_dotenv('.env') 

True

In [3]:
API_KEY = os.environ.get('NEW_YORK_TIMES_API_KEY')


In [21]:
def nytimes_query(
    
    api_key, query,
    glocations = None, headline = None, news_desk = None,
    organizations = None, persons = None, byline = None,
    subject = None, news_type = None, type_of_material = None,
    begin_date = None, end_date = None, n_page = 0
    ):
    # Set the base url for the query
    base_url = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={query}'

        # Empty dictionary for filters
    filter_queries = {}
    # empty dictionary for dates
    dates = {}
    # Populate the filter dictionary
    if glocations:
        filter_queries.update({'glocations': glocations})
        print(filter_queries)
    if headline:
        filter_queries.update({'headline': headline})
        print(filter_queries)
    if news_desk:
        filter_queries.update({'news_desk': news_desk})
        print(filter_queries)
    if organizations:
        filter_queries.update({'organizations': organizations})
        print(filter_queries)
    if persons:
        filter_queries.update({'persons': persons})
        print(filter_queries)
    if subject:
        filter_queries.update({'subject': subject})
        print(filter_queries)
    if byline:
        filter_queries.update({'byline': byline})
        print(filter_queries)
    if news_type:
        filter_queries.update({'news_type': news_type})
        print(filter_queries)
    if type_of_material:
        filter_queries.update({'type_of_material': type_of_material})
        print(filter_queries)
    # Populate the date dictionary
    if begin_date:
        dates.update({'begin_date': begin_date})
        print(dates)
    if end_date:
        dates.update({'end_date':end_date})
        print(dates)
    # If 1 filter is present, and/or date params, add to URL and execute query
    if len(filter_queries) == 0:
        if len(dates) == 1:
            base_url += f'&{list(dates.keys())[0]}={list(dates.values())[0]}'
            print(base_url)
        elif len(dates) == 2:
            base_url += '&'
            for i in dates.keys():
                base_url += f'{i}={dates[i]}&'
            print(base_url)
    elif len(filter_queries) == 1:
        base_url += f'&fq={list(filter_queries.keys())[0]}:("{list(filter_queries.values())[0]}")'
        print(base_url)
        if len(dates) == 1:
            base_url += f'&{list(dates.keys())[0]}={list(dates.values())[0]}'
            print(base_url)
        elif len(dates) == 2:
            base_url += '&'
            for i in dates.keys():
                base_url += f'{i}={dates[i]}&'
            print(base_url)
    # If 2 or more filters are present, concatenate with AND, add dates if present and execute
    elif  len(filter_queries) > 1:
        base_url += '&fq='
        for i in filter_queries.keys():
            base_url += f'{i}:({filter_queries[i]}) AND '
        # remove the last 'AND ' at the end of the loop
        base_url = base_url[:-5]
        print(base_url)

        if len(dates) == 1:
            base_url += f'&{list(filter_queries.keys())[0]}={list(filter_queries.values())[0]}&'
            print(base_url)

        elif len(dates) == 2:
            base_url += '&'
            print(base_url)

            for i in dates.keys():
                base_url += f'{i}={dates[i]}&'
                print(base_url)

    # concatenate page number and api key and make the request.
    # Returns a truncated JSON indexed past the metadata
    # If you want the full json, simply remove ['response']['docs']
    # from the return line
    base_url += f'&page={n_page}'
    base_url += f'&api-key={api_key}'
    print(f'Final query : {base_url}')
    r = requests.get(base_url)
    json_data = r.json()
    return r.json()['response']['docs']


In [40]:
#Nvidia,Meta,Tesla
# query = nytimes_query(API_KEY,query='Nvidia',news_desk='Business',begin_date='20220101',end_date='20220131')
query = nytimes_query(API_KEY,query='Tesla',begin_date='20220102',end_date='20220102')

query

{'begin_date': '20220102'}
{'begin_date': '20220102', 'end_date': '20220102'}
Final query : https://api.nytimes.com/svc/search/v2/articlesearch.json?q=Tesla&page=0&api-key=YPrMAd93hm7pmTqscIQVlnGZiv1EwSOI


[{'abstract': 'Tesla mechanics in Sweden have been striking for six months with little movement from their employer. Nordic shareholders hope to change that.',
  'web_url': 'https://www.nytimes.com/2024/06/13/business/elon-musk-tesla-vote-sweden-norway.html',
  'snippet': 'Tesla mechanics in Sweden have been striking for six months with little movement from their employer. Nordic shareholders hope to change that.',
  'lead_paragraph': 'The fight over Elon Musk’s pay package and the vote over whether to move Tesla’s headquarters to Texas have been the main focus of attention ahead of the company’s shareholders meeting on Thursday, but not for investors from Nordic countries.',
  'print_section': 'B',
  'print_page': '6',
  'source': 'The New York Times',
  'multimedia': [{'rank': 0,
    'subtype': 'xlarge',
    'caption': None,
    'credit': None,
    'type': 'image',
    'url': 'images/2024/06/13/multimedia/13tesla-labor-hcmg/13tesla-labor-hcmg-articleLarge.jpg',
    'height': 400,
   

In [41]:
tmp = pd.DataFrame(query)
tmp

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,headline,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,subsection_name
0,Tesla mechanics in Sweden have been striking f...,https://www.nytimes.com/2024/06/13/business/el...,Tesla mechanics in Sweden have been striking f...,The fight over Elon Musk’s pay package and the...,B,6,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Tesla’s Nordic Shareholders Seek to ...,"[{'name': 'organizations', 'value': 'Tesla Mot...",2024-06-13T04:00:33+0000,article,Business,Business Day,"{'original': 'By Melissa Eddy', 'person': [{'f...",News,nyt://article/cfa446ef-8922-5239-a5d4-d5dc3fa0...,901,nyt://article/cfa446ef-8922-5239-a5d4-d5dc3fa0...,NaN
1,Investors voted overwhelmingly to reinstate th...,https://www.nytimes.com/2024/06/14/business/de...,Investors voted overwhelmingly to reinstate th...,Elon Musk got his way.,NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'What Elon Musk’s Convincing Win Mean...,"[{'name': 'subject', 'value': 'internal-storyl...",2024-06-14T12:03:25+0000,article,Business,Business Day,"{'original': 'By Andrew Ross Sorkin, Ravi Matt...",News,nyt://article/3e8c7d76-361e-56f2-940b-de142300...,1900,nyt://article/3e8c7d76-361e-56f2-940b-de142300...,DealBook
2,"The vote on the compensation package, which he...",https://www.nytimes.com/interactive/2024/06/14...,"The vote on the compensation package, which he...","The vote on the compensation package, which he...",NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'How Tesla Shareholders Upheld Elon M...,"[{'name': 'organizations', 'value': 'Tesla Mot...",2024-06-14T14:01:23+0000,multimedia,Graphics,Business Day,{'original': 'By Lily Boyce and Jason Karaian'...,Interactive Feature,nyt://interactive/9abf3cb7-2a91-5b7a-95d8-2b20...,0,nyt://interactive/9abf3cb7-2a91-5b7a-95d8-2b20...,NaN
3,The vote was seen as a referendum on his manag...,https://www.nytimes.com/2024/06/13/business/te...,The vote was seen as a referendum on his manag...,Tesla shareholders have reaffirmed a pay award...,B,1,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Tesla Shareholders Approve Big Stock...,"[{'name': 'subject', 'value': 'Electric and Hy...",2024-06-13T21:13:08+0000,article,Business,Business Day,"{'original': 'By Jack Ewing and Peter Eavis', ...",News,nyt://article/87b604dc-b6be-50d3-a7b9-571a91e8...,952,nyt://article/87b604dc-b6be-50d3-a7b9-571a91e8...,NaN
4,The vote is seen as a referendum on the limits...,https://www.nytimes.com/2024/06/12/business/te...,The vote is seen as a referendum on the limits...,"Under Elon Musk’s leadership, Tesla popularize...",B,1,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Tesla Investors to Decide if Musk De...,"[{'name': 'persons', 'value': 'Musk, Elon', 'r...",2024-06-12T16:00:06+0000,article,Business,Business Day,"{'original': 'By Jack Ewing and Peter Eavis', ...",News,nyt://article/8ea0f150-7b73-5a16-accf-caa6e6f3...,1494,nyt://article/8ea0f150-7b73-5a16-accf-caa6e6f3...,NaN
5,About 72 percent of shares in the balloting af...,https://www.nytimes.com/2024/06/14/business/te...,About 72 percent of shares in the balloting af...,Tesla shareholders decisively backed a proposa...,B,3,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Musk’s Tesla Pay Package Got Big Mar...,"[{'name': 'persons', 'value': 'Musk, Elon', 'r...",2024-06-14T11:04:37+0000,article,Business,Business Day,{'original': 'By Jason Karaian and Jack Ewing'...,News,nyt://article/c9562780-0885-5f8f-a636-ba5b7bc8...,823,nyt://article/c9562780-0885-5f8f-a636-ba5b7bc8...,NaN
6,A huge run-up in the stock’s value followed a ...,https://www.nytimes.com/2024/06/13/business/te...,A huge run-up in the stock’s value followed a ...,The Tesla shareholder vote over Elon Mu